# Partitioned heat conduction
https://github.com/precice/tutorials/tree/master/partitioned-heat-conduction

## Summary
We solve a simple heat equation. The domain is partitioned and the coupling is established in a Dirichlet-Neumann fashion.

## Setup

We solve a partitioned heat equation. For information on the non-partitioned case, please refer to [1, p.37ff]. In this tutorial the computational domain is partitioned and coupled via preCICE. The coupling roughly follows the approach described in [2].

![Case setup of partitioned-heat-conduction case](images/tutorials-partitioned-heat-conduction-setup.png)

Case setup from [3]. `D` denotes the Dirichlet participant and `N` denotes the Neumann participant.

The heat equation is solved on a rectangular domain `Omega = [0,2] x [0,1]` with given Dirichlet boundary conditions. We split the domain at `x_c = 1` using a straight vertical line, the coupling interface. The left part of the domain will be referred to as the Dirichlet partition and the right part as the Neumann partition. To couple the two participants we use Dirichlet-Neumann coupling. Here, the Dirichlet participant receives Dirichlet boundary conditions (`Temperature`) at the coupling interface and solves the heat equation using these boundary conditions on the left part of the domain. Then the Dirichlet participant computes the resulting heat flux (`Flux`) from the solution and sends it to the Neumann participant. The Neumann participant uses the flux as a Neumann boundary condition to solve the heat equation on the right part of the domain. We then extract the temperature from the solution and send it back to the Dirichlet participant. This establishes the coupling between the two participants.

This simple case allows us to compare the solution for the partitioned case to a known analytical solution (method of manufactures solutions, see [1, p.37ff]). For more usage examples and details, please refer to [3, sect. 4.1].

In [1]:
import argparse
import numpy as np
import multiprocessing

In [2]:
from __future__ import print_function, division
from enum import Enum
from fenics import Function, FunctionSpace, Expression, Constant, DirichletBC, TrialFunction, TestFunction, \
    File, solve, lhs, rhs, grad, inner, assemble, dot, dx, ds, project, sqrt, interpolate, VectorFunctionSpace, \
    MeshFunction, MPI, SubDomain, Point, RectangleMesh, near, Function, VectorFunctionSpace, Expression
from fenicsprecice import Adapter
import dolfin
from dolfin import FacetNormal, dot

In [3]:
def run_precice(problem, domain_part):
    mesh, coupling_boundary, remaining_boundary = get_geometry(domain_part)

    # Define function space using mesh
    V = FunctionSpace(mesh, 'P', 2)
    V_g = VectorFunctionSpace(mesh, 'P', 1)
    W = V_g.sub(0).collapse()

    # Define boundary conditions
    u_D = Expression('1 + x[0]*x[0] + alpha*x[1]*x[1] + beta*t', degree=2, alpha=alpha, beta=beta, t=0)
    u_D_function = interpolate(u_D, V)

    if problem is ProblemType.DIRICHLET:
        # Define flux in x direction
        f_N = Expression("2 * x[0]", degree=1, alpha=alpha, t=0)
        f_N_function = interpolate(f_N, W)

    # Define initial value
    u_n = interpolate(u_D, V)
    u_n.rename("Temperature", "")

    precice, precice_dt, initial_data = None, 0.0, None

    # Initialize the adapter according to the specific participant
    if problem is ProblemType.DIRICHLET:
        precice = Adapter(adapter_config_filename="/home/jovyan/fenics/precice-adapter-config-D.json")
        precice_dt = precice.initialize(coupling_boundary, read_function_space=V, write_object=f_N_function)
    elif problem is ProblemType.NEUMANN:
        precice = Adapter(adapter_config_filename="/home/jovyan/fenics/precice-adapter-config-N.json")
        precice_dt = precice.initialize(coupling_boundary, read_function_space=W, write_object=u_D_function)

    dt = Constant(0)
    dt.assign(np.min([fenics_dt, precice_dt]))

    # Define variational problem
    u = TrialFunction(V)
    v = TestFunction(V)
    f = Expression('beta - 2 - 2*alpha', degree=2, alpha=alpha, beta=beta, t=0)
    F = u * v / dt * dx + dot(grad(u), grad(v)) * dx - (u_n / dt + f) * v * dx

    bcs = [DirichletBC(V, u_D, remaining_boundary)]

    # Set boundary conditions at coupling interface once wrt to the coupling
    # expression
    coupling_expression = precice.create_coupling_expression()
    if problem is ProblemType.DIRICHLET:
        # modify Dirichlet boundary condition on coupling interface
        bcs.append(DirichletBC(V, coupling_expression, coupling_boundary))
    if problem is ProblemType.NEUMANN:
        # modify Neumann boundary condition on coupling interface, modify weak
        # form correspondingly
        F += v * coupling_expression * dolfin.ds

    a, L = lhs(F), rhs(F)

    # Time-stepping
    u_np1 = Function(V)
    u_np1.rename("Temperature", "")
    t = 0

    # reference solution at t=0
    u_ref = interpolate(u_D, V)
    u_ref.rename("reference", " ")

    # mark mesh w.r.t ranks
    mesh_rank = MeshFunction("size_t", mesh, mesh.topology().dim())
    if problem is ProblemType.NEUMANN:
        mesh_rank.set_all(MPI.rank(MPI.comm_world) + 4)
    else:
        mesh_rank.set_all(MPI.rank(MPI.comm_world) + 0)
    mesh_rank.rename("myRank", "")

    # Generating output files
    temperature_out = File("out/%s.pvd" % precice.get_participant_name())
    ref_out = File("out/ref%s.pvd" % precice.get_participant_name())
    error_out = File("out/error%s.pvd" % precice.get_participant_name())
    ranks = File("out/ranks%s.pvd" % precice.get_participant_name())

    # output solution and reference solution at t=0, n=0
    n = 0
    print('output u^%d and u_ref^%d' % (n, n))
    temperature_out << u_n
    ref_out << u_ref
    ranks << mesh_rank

    error_total, error_pointwise = compute_errors(u_n, u_ref, V)
    error_out << error_pointwise

    # set t_1 = t_0 + dt, this gives u_D^1
    # call dt(0) to evaluate FEniCS Constant. Todo: is there a better way?
    u_D.t = t + dt(0)
    f.t = t + dt(0)

    if problem is ProblemType.DIRICHLET:
        flux = Function(V_g)
        flux.rename("Flux", "")

    while precice.is_coupling_ongoing():

        # write checkpoint
        if precice.is_action_required(precice.action_write_iteration_checkpoint()):
            precice.store_checkpoint(u_n, t, n)

        read_data = precice.read_data()

        # Update the coupling expression with the new read data
        precice.update_coupling_expression(coupling_expression, read_data)

        dt.assign(np.min([fenics_dt, precice_dt]))

        # Compute solution u^n+1, use bcs u_D^n+1, u^n and coupling bcs
        solve(a == L, u_np1, bcs)

        # Write data to preCICE according to which problem is being solved
        if problem is ProblemType.DIRICHLET:
            # Dirichlet problem reads temperature and writes flux on boundary to Neumann problem
            determine_gradient(V_g, u_np1, flux)
            flux_x = interpolate(flux.sub(0), W)
            precice.write_data(flux_x)
        elif problem is ProblemType.NEUMANN:
            # Neumann problem reads flux and writes temperature on boundary to Dirichlet problem
            precice.write_data(u_np1)

        precice_dt = precice.advance(dt(0))

        # roll back to checkpoint
        if precice.is_action_required(precice.action_read_iteration_checkpoint()):
            u_cp, t_cp, n_cp = precice.retrieve_checkpoint()
            u_n.assign(u_cp)
            t = t_cp
            n = n_cp
        else:  # update solution
            u_n.assign(u_np1)
            t += float(dt)
            n += 1

        if precice.is_time_window_complete():
            u_ref = interpolate(u_D, V)
            u_ref.rename("reference", " ")
            error, error_pointwise = compute_errors(u_n, u_ref, V, total_error_tol=error_tol)
            print('n = %d, t = %.2f: L2 error on domain = %.3g' % (n, t, error))
            # output solution and reference solution at t_n+1
            print('output u^%d and u_ref^%d' % (n, n))
            temperature_out << u_n
            ref_out << u_ref
            error_out << error_pointwise

        # Update Dirichlet BC
        u_D.t = t + float(dt)
        f.t = t + float(dt)

    # Hold plot
    precice.finalize()

In [4]:
class ProblemType(Enum):
    """
    Enum defines problem type. Details see above.
    """
    DIRICHLET = 1  # Dirichlet problem
    NEUMANN = 2  # Neumann problem

In [5]:
class DomainPart(Enum):
    """
    Enum defines which part of the domain [x_left, x_right] x [y_bottom, y_top] we compute.
    """
    LEFT = 1  # left part of domain in simple interface case
    RIGHT = 2  # right part of domain in simple interface case
    CIRCULAR = 3  # circular part of domain in complex interface case
    RECTANGLE = 4  # domain excluding circular part of complex interface case

In [6]:
def determine_gradient(V_g, u, flux):
    """
    compute flux following http://hplgit.github.io/INF5620/doc/pub/fenics_tutorial1.1/tu2.html#tut-poisson-gradu
    :param V_g: Vector function space
    :param u: solution where gradient is to be determined
    :param flux: returns calculated flux into this value
    """

    w = TrialFunction(V_g)
    v = TestFunction(V_g)

    a = inner(w, v) * dx
    L = inner(grad(u), v) * dx
    solve(a == L, flux)

In [7]:
def compute_errors(u_approx, u_ref, v, total_error_tol=10 ** -4):
    # compute pointwise L2 error
    error_normalized = (u_ref - u_approx) / u_ref
    # project onto function space
    error_pointwise = project(abs(error_normalized), v)
    # determine L2 norm to estimate total error
    error_total = sqrt(assemble(inner(error_pointwise, error_pointwise) * dx))
    error_pointwise.rename("error", " ")

    assert (error_total < total_error_tol)

    return error_total, error_pointwise

In [8]:
class ExcludeStraightBoundary(SubDomain):
    def get_user_input_args(self, args):
        self._interface = args.interface

    def inside(self, x, on_boundary):
        tol = 1E-14
        if on_boundary and not near(x[0], x_coupling, tol) or near(x[1], y_top, tol) or near(x[1], y_bottom, tol):
            return True
        else:
            return False



In [9]:
class StraightBoundary(SubDomain):
    def inside(self, x, on_boundary):
        tol = 1E-14
        if on_boundary and near(x[0], x_coupling, tol):
            return True
        else:
            return False

In [10]:
def get_geometry(domain_part):
    nx = ny = 9
    low_resolution = 5
    high_resolution = 5
    n_vertices = 20

    if domain_part is DomainPart.LEFT:
        p0 = Point(x_left, y_bottom)
        p1 = Point(x_coupling, y_top)
    elif domain_part is DomainPart.RIGHT:
        p0 = Point(x_coupling, y_bottom)
        p1 = Point(x_right, y_top)
    else:
        raise Exception("invalid domain_part: {}".format(domain_part))

    mesh = RectangleMesh(p0, p1, nx, ny, diagonal="left")
    coupling_boundary = StraightBoundary()
    remaining_boundary = ExcludeStraightBoundary()

    return mesh, coupling_boundary, remaining_boundary

In [11]:
y_bottom, y_top = 0, 1
x_left, x_right = 0, 2
x_coupling = 1.0  # x coordinate of coupling interface
radius = 0.2
midpoint = Point(0.5, 0.5)

In [12]:
fenics_dt = .1  # time step size
# Error is bounded by coupling accuracy. In theory we would obtain the analytical solution.
error_tol = 10e-3

In [13]:
alpha = 3  # parameter alpha
beta = 1.3  # parameter beta

In [14]:
p_1 = multiprocessing.Process(target=run_precice, args=(ProblemType.DIRICHLET, DomainPart.LEFT))

In [15]:
p_2 = multiprocessing.Process(target=run_precice, args=(ProblemType.NEUMANN, DomainPart.RIGHT))

In [16]:
p_1.start()

output u^0 and u_ref^0
n = 1, t = 0.10: L2 error on domain = 7.27e-09
output u^1 and u_ref^1
n = 2, t = 0.20: L2 error on domain = 5.06e-10
output u^2 and u_ref^2
n = 3, t = 0.30: L2 error on domain = 4.75e-11
output u^3 and u_ref^3
n = 4, t = 0.40: L2 error on domain = 1.3e-11
output u^4 and u_ref^4
n = 5, t = 0.50: L2 error on domain = 4.95e-11
output u^5 and u_ref^5
n = 6, t = 0.60: L2 error on domain = 8.57e-12
output u^6 and u_ref^6
n = 7, t = 0.70: L2 error on domain = 1.52e-11
output u^7 and u_ref^7
n = 8, t = 0.80: L2 error on domain = 1.6e-11
output u^8 and u_ref^8
n = 9, t = 0.90: L2 error on domain = 6.42e-12
output u^9 and u_ref^9
n = 10, t = 1.00: L2 error on domain = 8.8e-12
output u^10 and u_ref^10


In [17]:
p_2.start()

output u^0 and u_ref^0---[precice]  This is preCICE version 2.3.0
---[precice]  Revision info: no-info [Git failed/Not a repository]
---[precice]  Configuration: Debug
---[precice]  Configuring preCICE with configuration "/home/jovyan/precice-config.xml"
---[precice]  I am participant "Dirichlet"
---[precice]  Setting up master communication to coupling partner/s
---[precice]  This is preCICE version 2.3.0
---[precice]  Revision info: no-info [Git failed/Not a repository]
---[precice]  Configuration: Debug
---[precice]  Configuring preCICE with configuration "/home/jovyan/precice-config.xml"
---[precice]  I am participant "Neumann"
---[precice]  Setting up master communication to coupling partner/s
---[precice]  Masters are connected
---[precice]  Setting up preliminary slaves communication to coupling partner/s
---[precice]  Prepare partition for mesh Dirichlet-Mesh
---[precice]  Masters are connected
---[precice]  Setting up preliminary slaves communication to coupling partner/s
---[